# Solution for Neutrality

_This challenge was inspired by, and initially required, statistical analysis (where you are given hundreds of thousands of lines). Then I realised 319 lines is more than sufficient for a 320-bit message. Then I realised we could push this down even  more, and so we end up with the 200 seen in this final version._

In [1]:
from pwn import *
from sage.all import *
from Crypto.Util.number import long_to_bytes, bytes_to_long
from tqdm import tqdm, trange

First, let's load all our data from `output.txt` into an array, and print the number of rows and length of the flag in bits.

In [2]:
nums = [int(x) for x in open('output.txt').read().splitlines()]
bits = max(n.bit_length() for n in nums)
len(nums), bits

(200, 320)

We have 200 rows and 320 bits per row (i.e. 40 bytes). We want to find a 320-bit flag such that XORing it with each row produces a bitstream with exactly 160 1s and 160 0s.

The usual way of representing the vectors as elements of $\mathbb{Z}_2^{320}$ makes it really difficult to do such a bitcount. Instead, we change the bits to $\pm1$ instead, so that an XOR is just a pointwise multiplication, and a bitstream with exactly 160 1s and 160 0s is just a vector whose elements sum to zero.

In other words, if $M$ is the 200x320 matrix of $\pm1$s corresponding to `output.txt`, then the flag we seek is simply an element in the kernel of $M$ whose coordinates are all in $\pm1$.

With that in mind, here's a few mathematical notes:
- There should be at least at least two answers: the flag and its complement.
- If we were given at least 319 rows instead of merely 200, we'd have a 1-dimensional kernel from which we can extract our flag. As it stands, our kernel will have dimension $320-200=120$, which is quite a huge space to search.
- Quadratic linearisation is a thing. What this means is that instead of 320-dimensional vectors representing $b_0$ to $b_{319}$, we instead have 51040-dimensional vectors representing all all possible products $b_i b_j, i \neq j$. Since each single row can be expanded to 320 equations, what we really have is a system of $320*200=64000$ linear equations of 51040 variables, which should lead us to a unique solution. However, even though this has polynomial-time complexity, we don't actual know of a way to make this run in a reasonable time. It will be interesting to see if any teams managed to make this approach work.

The approach we use will in fact be to find the flag in the 120-dimensional kernel. By reducing the basis of the kernel to reduced row echelon form, this gives us a representation of the $2^{120}$ vectors whose first 120 coefficients are all $\pm1$.

Well, I say "first 120", but really we can pick any 120 coefficients by rearranging the bits/columns as we wish. This is useful because we can assume quite a few things about the flag:
1. It begins with `SEE{` and ends with `}`. That's 5 bytes (40 bits) right there.
2. The inner 35 bytes of the flag are likely ASCII and will have their highest bit cleared. So that's another 35 bits.

Those reductions mean we only need to brute-force a sample space of $2^{45}$ vectors. Even then we don't need to look at the entirety of the vector, it suffices initially to check whether the 121st coefficient is $\pm1$ (and only if that succeeds then checking the remaining coefficients). But this is exactly the Subset Sum Problem, which thus reduces to roughly $2^{23}$ via a [Meet-in-the-middle attack](https://en.wikipedia.org/wiki/Meet-in-the-middle_attack). This is certainly a brute-forceable range.

---

In our implementation, we XOR every row with the bits we know to be set. This just means that we don't need to keep track of which known bits were set, since we are rearranging the columns. We will let the first 75 columns correspond to the known bits (which are now all 0 bits), and the remaining 45 are the ones we wish to brute-force.

In [3]:
mask = b'SEE{' + b'\0'*35 + b'}'
mask_long = bytes_to_long(mask)
M = Matrix(QQ,[[(-1)**((mask_long^n)>>i) for i in range(320)] for n in nums])

arr = sorted(range(320), key=lambda n:n % 8 != 7 and 8 <= n < bits - 32)
M = matrix([M.column(i) for i in arr])
B = matrix(M.left_kernel().basis())
B

120 x 320 dense matrix over Rational Field

As expected, we have a 120-dimensional kernel. The basis B is already row-reduced, which is handy for us. Let's just quickly make sure that we do have the identity matrix in the left part of B:

In [4]:
B[:120,:120]==1

True

At this point, we decide to make the shift to double-precision floats rather than `QQ` (exact fractions). This is a trade-off we've chosen to make, where we get much faster addition and subtraction, but lose out on the ability to calculate an exact value of `1` or `-1`. Also, we get to use numpy, which is a win.

To start off with, we define a function to get all $2^n$ combinations of adding or subtracting each element from a vector of length $n$. For example, the set of all combinations of `vec=[a,b,c]` should be `[a+b+c,a+b-c,a-b+c,a-b-c,-a+b+c,-a+b-c,-a-b+c,-a-b-c]`. We speed this up by using a [Gray code](https://en.wikipedia.org/wiki/Gray_code) to only flip one bit at at a time.

In [5]:
import numpy as np
def get_all_sums(vec):
    MAXLEN = 2**len(vec)
    result = np.empty(MAXLEN)
    n = 0
    s = vec.sum()
    for q in trange(MAXLEN):
        result[n] = s
        graybit = (~q&-~q).bit_length()-1
        if q < MAXLEN-1:
            n ^= 1 << graybit
            s += 2 * vec[graybit] * (-1)**(n>>graybit)
    return result

So all the ingredients are there. We have a 120x320 matrix B, and we want to find a $\pm1$ combinations of vectors in B such that every coefficient is $\pm1$. We know the first 75 bits are cleared, which leaves a subset-sum problem on the remaining 45 bits. We will split this into size 23 and 22 respectively.

In [6]:
Bnp = B.numpy()
B0, B1, B2 = np.split(Bnp[:,120], [75, -22]) # splits the rows into a 75/23/22 split
print(len(B0), len(B1), len(B2))
B1s = get_all_sums(B1)
B2s = get_all_sums(B2) 

75 23 22


100%|██████████| 4194304/4194304 [00:05<00:00, 833369.58it/s]


Now, if we had chosen to stay in `QQ`, we would get an exact match in these two sets, but since we're working in floats we need a correction term. This is exacerbated by our use of the Gray code, which causes the rounding errors to accumulate, but we pick a reasonable $\varepsilon = 10^{-8}$ anyway, so that we need to find matches of the target to within $\varepsilon$. Essentially, this means that our choice of data structure needs to be a sorted set rather than a hashtable.

In [7]:
EPSILON = 1e-8
order = np.argsort(B1s, 0)
mitm = B1s[order] - B0.sum()
pts = np.searchsorted(mitm, np.stack((-1-B2s-EPSILON,-1-B2s+EPSILON,1-B2s-EPSILON,1-B2s+EPSILON)).T)

Ok, so recall that we want the sum to be either 1 or -1. Taking a window of $\varepsilon$ on either side gives 4 bisection points for each value. Now we just test all these matches to see whether they extend to a vector in which _every_ coefficient is $\pm1$, and those will be our final flag candidates!

In [8]:
for q, (t0,t1,t2,t3) in enumerate(tqdm(pts)):
    for b in (*range(t0,t1),*range(t2,t3)):
        n = q * len(mitm) + order[b]
        v = sum(a * (-1)**(n>>i) for i,a in enumerate(Bnp[75:])) - sum(Bnp[:75],0)
        if all(abs(x * x - 1) < EPSILON for x in v):
            soln = (v + 0.5).astype('int')
            flag = long_to_bytes(int(''.join(str(b) for _,b in sorted(zip(arr,soln)))[::-1],2) ^ mask_long)
            print(f'Possible flag: {flag}')

 92%|█████████▏| 3867460/4194304 [00:06<00:00, 608948.88it/s]

Possible flag: b'SEE{50-50_can_be_leaky_4c17bf2a20c4a8df}'


100%|██████████| 4194304/4194304 [00:06<00:00, 603729.18it/s]
